## Project Motivation

The primary aim of this project is to delve deeply into the weekly sales data of a bustling coffee shop to uncover underlying patterns and behaviors. By leveraging sophisticated data analytics techniques, we seek to dissect the nuances of weekly revenue fluctuations and customer purchasing behaviors. The ultimate goal is to segment customers into distinct groups based on their spending patterns and interaction frequency. This strategic segmentation will empower us to develop targeted marketing tactics tailored specifically to each customer group. By understanding when and how customers engage with our offerings, we can design personalized promotions, optimize our inventory management, and ultimately enhance customer satisfaction and loyalty. This data-driven approach not only aims to boost the shop's profitability but also enhances the overall customer experience by aligning our services more closely with customer needs and preferences.


## Libraries

In [112]:
import pandas as pd
import numpy as np
import altair as alt
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

## Data 

In [99]:
# Import Data
coffee_df = pd.read_excel('CoffeeShop.xlsx')
coffee_df.head()

,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out
0,1,ORD001,2024-02-12 07:04:00,It008,1,Alex,out
1,2,ORD002,2024-02-12 07:09:00,It014,1,Jordan,in
2,3,ORD003,2024-02-12 07:14:00,It008,1,Taylor,out
3,4,ORD004,2024-02-12 07:18:00,It019,1,Casey,out
4,5,ORD005,2024-02-12 07:23:00,It024,1,Jamie,out


In [110]:
# Import Data for Merging
items_df = pd.read_csv('items.csv')
items_df.head()

,item_id,sku,item_name,item_cat,item_size,item_price
0,It001,HDR-CAP-MD,Cappuccino,Hot Drinks,Medium,3.45
1,It002,HDR-CAP-LG,Cappuccino,Hot Drinks,Large,3.75
2,It003,HDR-LAT-MD,Latte,Hot Drinks,Medium,3.45
3,It004,HDR-LAT-LG,Latte,Hot Drinks,Large,3.75
4,It005,HDR-FLT,Flat White,Hot Drinks,NaN,3.15


In [ ]:
# Merge 
df = coffee_df.merge(items_df, on='item_id')

In [102]:
df.head()

,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out,sku,item_name,item_cat,item_size,item_price
0,1,ORD001,2024-02-12 07:04:00,It008,1,Alex,out,HDR-ESP,Espresso,Hot Drinks,NaN,2.15
1,3,ORD003,2024-02-12 07:14:00,It008,1,Taylor,out,HDR-ESP,Espresso,Hot Drinks,NaN,2.15
2,32,ORD023,2024-02-12 08:50:00,It008,1,Alex,out,HDR-ESP,Espresso,Hot Drinks,NaN,2.15
3,79,ORD060,2024-02-12 13:11:00,It008,1,Brooke,out,HDR-ESP,Espresso,Hot Drinks,NaN,2.15
4,125,ORD097,2024-02-13 07:50:00,It008,1,Jamie,in,HDR-ESP,Espresso,Hot Drinks,NaN,2.15


In [103]:
# Check Date Range
date_min = df['created_at'].min()
date_max = df['created_at'].max()
print(f"The date range of the dataset is from {date_min} to {date_max}.")

The date range of the dataset is from 2024-02-12 07:04:00 to 2024-02-17 16:59:00.


## Item Pricing and Total Sold
The **Item Pricing Bar Chart** provides a clear visual representation of the prices of different items available at the coffee shop. Each bar represents a unique item, with its height indicating the price. This visualization helps in quickly identifying the range of prices across different products, aiding in pricing strategy adjustments and inventory management.

In [104]:
# Bar Chart
df['item_price'] = df['item_price'].astype(float)

# Create a bar chart
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('item_name:N', sort='-y', title='Item Name'),
    y=alt.Y('item_price:Q', title='Price ($)'),
    tooltip=[alt.Tooltip('item_name:N', title='Item Name'), alt.Tooltip('item_price:Q', title='Price ($)')]
).properties(
    title='Item Pricing',
    width=600,
    height=300
)
chart

alt.Chart(...)

## Category Peformance
The **Category Performance Pie Chart** illustrates the distribution of total revenue generated by each item category. This pie chart makes it easy to see which categories are the most profitable and which ones may require promotional strategies to boost sales. Understanding how different categories contribute to overall revenue helps in making informed decisions about product placement, promotions, and purchasing.

In [105]:
category_revenue = df.groupby('item_cat')['item_price'].sum().reset_index()
category_revenue_json = category_revenue.to_json(orient='records')

# Altair plot with legend
chart = alt.Chart(category_revenue).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field='item_price', type='quantitative', stack=True),
    color=alt.Color(field='item_cat', type='nominal', legend=alt.Legend(title='Categories')),
    tooltip=[alt.Tooltip('item_cat:N', title='Category'), alt.Tooltip('item_price:Q', title='Revenue')]
).properties(
    title='Revenue by Item Category',
    width=400,
    height=400
)
chart

alt.Chart(...)

## Weekly Revenue
The **Weekly Revenue Line Chart** shows the fluctuations in revenue over time, providing insights into weekly sales performance. This trend line is crucial for spotting patterns in sales, such as weekly highs and lows, helping to forecast future sales and plan for inventory accordingly. It also aids in understanding the impact of marketing campaigns and seasonal variations on sales.

In [106]:
# Convert 'created_at' to datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# Calculate revenue for each row
df['revenue'] = df['quantity'] * df['item_price']

# Set 'created_at' as the index
df.set_index('created_at', inplace=True)

# Group by week and sum the revenue
weekly_revenue = df.resample('W').revenue.sum()

# Printing the weekly revenue in a formal manner
print("Weekly Revenue Summary:")
print("-----------------------")
for date, revenue in weekly_revenue.items():
    print(f"Week ending {date.strftime('%Y-%m-%d')}: ${revenue:,.2f}")

Weekly Revenue Summary:
-----------------------
Week ending 2024-02-18: $1,857.45


In [107]:
# Calculate daily revenue
daily_revenue = df.resample('D').revenue.sum().reset_index()

# Create an interactive line chart
line_chart = alt.Chart(daily_revenue).mark_line(point=True).encode(
    x='created_at:T',
    y=alt.Y('revenue:Q', title='Revenue'),
    tooltip=['created_at:T', 'revenue:Q']
).properties(
    width=600,
    height=300,
    title='Daily Revenue Trend'
).interactive()

line_chart

alt.Chart(...)

## Data Visualization Summary
These visualizations collectively offer a comprehensive overview of the financial health of the business, highlighting key areas that require attention and those that are performing well. By analyzing these trends and distributions, the business can tailor its strategies to optimize profitability and customer satisfaction.

## Customer Segmentation Analysis

In [113]:
def simple_kmeans(data, n_clusters, n_init=10, max_iter=300):
    best_inertia = np.inf
    best_centers = None
    
    for _ in range(n_init):
        # Randomly choose clusters
        indices = np.random.choice(data.shape[0], n_clusters, replace=False)
        centers = data[indices]

        for _ in range(max_iter):
            # Assign clusters
            distances = np.sqrt(((data - centers[:, np.newaxis])**2).sum(axis=2))
            closest = np.argmin(distances, axis=0)

            # Calculate new centers
            new_centers = np.array([data[closest == k].mean(axis=0) for k in range(n_clusters)])

            # Check for convergence
            if np.all(new_centers == centers):
                break

            centers = new_centers

        # Calculate inertia
        inertia = sum(((data[closest == k] - centers[k])**2).sum() for k in range(n_clusters))

        if inertia < best_inertia:
            best_inertia = inertia
            best_centers = centers

    return best_centers, closest

# Assuming 'features_scaled' is already prepared
centers, labels = simple_kmeans(features_scaled, n_clusters=3)
print("Cluster centers:\n", centers)

Cluster centers:
 [[ 0.11030337  0.10576762]
 [-0.4084657  -0.40830505]
 [ 2.92143708  2.92927281]]


In [114]:
# Apply K-means clustering with the chosen number of clusters (k=3)
kmeans = KMeans(n_clusters=3, random_state=42)
customer_data['cluster'] = kmeans.fit_predict(features_scaled)

# Analyze cluster centers and segment sizes
cluster_centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=['total_spend', 'transaction_count'])
cluster_sizes = customer_data['cluster'].value_counts().rename('size')

# Join cluster centers with their sizes for a complete analysis
cluster_analysis = cluster_centers.join(cluster_sizes)

# Display cluster characteristics and their sizes
cluster_analysis

,total_spend,transaction_count,size
0,7.295200,1.584,125
1,81.656250,16.625,8
2,48.716667,10.000,6


### Overview of Customer Clusters
- **Cluster 0** (Low-Spending Occasional Visitors):
  - **Total Spend**: Approximately $7.30 per customer
  - **Transaction Count**: About 1.58 transactions per customer
  - **Size**: 125 customers
  - **Characteristics**: This cluster represents customers who spend little and visit occasionally. They might need more engagement to increase their spending and visit frequency.

- **Cluster 1** (High-Spending Loyal Customers):
  - **Total Spend**: Approximately $81.66 per customer
  - **Transaction Count**: About 16.63 transactions per customer
  - **Size**: 8 customers
  - **Characteristics**: These are loyal customers who visit frequently and spend a significant amount per visit. They are crucial to the business due to their high engagement and spending.

- **Cluster 2** (Moderate Spend Regulars):
  - **Total Spend**: Approximately $48.72 per customer
  - **Transaction Count**: About 10 transactions per customer
  - **Size**: 6 customers
  - **Characteristics**: These customers are regular visitors with moderate spending. They visit often but spend less per visit compared to Cluster 1.

### Insights and Actions
- **For Cluster 0**: Implement targeted promotions to increase their frequency of visits and spending. Consider loyalty programs or special offers that encourage more frequent purchases.
- **For Cluster 1**: Continue to enhance satisfaction and retain these customers through loyalty programs and personalized offers. Their high loyalty and spending are vital, so maintaining a good relationship is essential.
- **For Cluster 2**: Provide incentives that could help increase their spending levels or transaction frequency. Special offers, product recommendations, and upselling strategies could be effective for moving them towards higher spending behaviors similar to Cluster 1.

This segmentation allows us to tailor marketing strategies and operational adjustments effectively, aiming to enhance customer engagement and profitability across different customer groups.
